# Aprenentatge Supervisat - Regressions

**Descripció**

Anem a practicar i a familiaritzar-nos amb regressions

## Nivell 1

### Exercici 1
Crea almenys tres models de regressió diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.

### Exercici 2
Compara’ls en base al MSE i al R2 .

### Exercici 3
Entrena’ls utilitzant els diferents paràmetres que admeten.

### Exercici 4
Compara el seu rendiment utilitzant l’aproximació traint/test o utilitzant totes les dades (validació interna)

## Nivell 2
### Exercici 5
Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

## Nivell 3
### Exercici 6
No utilitzis la variable DepDelay a l’hora de fer prediccions